In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from keras import backend as K

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;


modelPath = '../models/VGG19_opt/run_adam.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="0"

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)
a = keras.optimizers.Adam(lr=.4);
print(a.lr)

<tf.Variable 'Adam/lr:0' shape=() dtype=float32_ref>


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [21]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values[:10]):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [16]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
#     with tf.device('/device:GPU:0'):
    base_model = VGG19(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    # Add a new top layer
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(0.7)(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)):
        base_model.layers[i].trainable = False

    # optimization
    lr={{uniform(0.00001,0.01)}};
    beta_1={{uniform(0.8,0.999)}};
    beta_2={{uniform(0.99,0.99999)}};
    decay={{uniform(0,0.01)}};

    print()
    print('lr=',lr);
    print('beta_1=',beta_1);
    print('beta_2=',beta_2);
    print('decay=',decay);
    print()

    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));

    print('Tensorboard NOT activated')

    model.fit(X_train, Y_train,
      epochs=100,
      batch_size = 64,
      validation_data=(X_valid, Y_valid),
      verbose=1,
      callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [17]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='VGG19Optimization_adam')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications.vgg19 import VGG19
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import cv2
except:
    pass

try:
    from subpr

100%|██████████| 10222/10222 [00:31<00:00, 325.23it/s]


Splitting into training/validation
Creating model

lr= 0.004376791431724103
beta_1= 0.921564385546966
beta_2= 0.9973643266762406
decay= 0.006517968154887782

Tensorboard NOT activated
Train on 7155 samples, validate on 3067 samples
Epoch 1/100
7155/7155 [==============================] - 30s 4ms/step - loss: 4.8475 - acc: 0.0091 - val_loss: 4.7859 - val_acc: 0.0114
Epoch 2/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.7391 - acc: 0.0177 - val_loss: 4.6365 - val_acc: 0.0241
Epoch 3/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.5565 - acc: 0.0333 - val_loss: 4.4187 - val_acc: 0.0470
Epoch 4/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.3568 - acc: 0.0537 - val_loss: 4.2446 - val_acc: 0.0727
Epoch 5/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.2304 - acc: 0.0693 - val_loss: 4.1663 - val_acc: 0.0799
Epoch 6/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.1395 - a

7155/7155 [==============================] - 27s 4ms/step - loss: 4.4156 - acc: 0.0528 - val_loss: 4.3865 - val_acc: 0.0577
Epoch 9/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.3531 - acc: 0.0574 - val_loss: 4.3468 - val_acc: 0.0613
Epoch 10/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.3237 - acc: 0.0555 - val_loss: 4.3048 - val_acc: 0.0668
Epoch 11/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.2939 - acc: 0.0609 - val_loss: 4.2699 - val_acc: 0.0652
Epoch 12/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.2639 - acc: 0.0653 - val_loss: 4.2413 - val_acc: 0.0740
Epoch 13/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.2414 - acc: 0.0660 - val_loss: 4.2229 - val_acc: 0.0753
Epoch 14/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.2110 - acc: 0.0635 - val_loss: 4.2117 - val_acc: 0.0701
Epoch 15/100
7155/7155 [==============================]

Epoch 68/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.9175 - acc: 0.1113 - val_loss: 3.9921 - val_acc: 0.1050
Epoch 69/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.9260 - acc: 0.1115 - val_loss: 3.9945 - val_acc: 0.1056
Epoch 70/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.9296 - acc: 0.1061 - val_loss: 3.9900 - val_acc: 0.1030
Epoch 71/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.9317 - acc: 0.1108 - val_loss: 3.9892 - val_acc: 0.1040
Epoch 72/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.9233 - acc: 0.1093 - val_loss: 3.9900 - val_acc: 0.1034
Epoch 73/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.9257 - acc: 0.1160 - val_loss: 3.9867 - val_acc: 0.1053
Epoch 74/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.9162 - acc: 0.1104 - val_loss: 3.9840 - val_acc: 0.1063
Epoch 75/100
7155/7155 [=================

...

7155/7155 [==============================] - 27s 4ms/step - loss: 3.1780 - acc: 0.2472 - val_loss: 3.8163 - val_acc: 0.1451
Epoch 52/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.1959 - acc: 0.2492 - val_loss: 3.8134 - val_acc: 0.1480
Epoch 53/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.1547 - acc: 0.2537 - val_loss: 3.8215 - val_acc: 0.1510
Epoch 54/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.1686 - acc: 0.2474 - val_loss: 3.8152 - val_acc: 0.1415
Epoch 55/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.1510 - acc: 0.2549 - val_loss: 3.8240 - val_acc: 0.1405
Epoch 56/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.1415 - acc: 0.2457 - val_loss: 3.8194 - val_acc: 0.1519
Epoch 57/100
7155/7155 [==============================] - 27s 4ms/step - loss: 3.1378 - acc: 0.2570 - val_loss: 3.8179 - val_acc: 0.1402
Epoch 58/100
7155/7155 [==============================

7155/7155 [==============================] - 28s 4ms/step - loss: 4.8732 - acc: 0.0103 - val_loss: 4.7668 - val_acc: 0.0121
Epoch 2/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.7258 - acc: 0.0179 - val_loss: 4.6131 - val_acc: 0.0241
Epoch 3/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.5386 - acc: 0.0340 - val_loss: 4.4195 - val_acc: 0.0515
Epoch 4/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.3667 - acc: 0.0488 - val_loss: 4.2430 - val_acc: 0.0672
Epoch 5/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.2516 - acc: 0.0623 - val_loss: 4.1499 - val_acc: 0.0779
Epoch 6/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.1857 - acc: 0.0679 - val_loss: 4.1059 - val_acc: 0.0825
Epoch 7/100
7155/7155 [==============================] - 27s 4ms/step - loss: 4.1148 - acc: 0.0804 - val_loss: 4.0527 - val_acc: 0.0871
Epoch 8/100
7155/7155 [==============================] - 27s

100%|██████████| 10222/10222 [00:49<00:00, 206.29it/s]


Splitting into training/validation


FailedPreconditionError: Attempting to use uninitialized value block1_conv1_7/kernel
	 [[Node: block1_conv1_7/kernel/read = Identity[T=DT_FLOAT, _class=["loc:@block1_conv1_7/kernel"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv1_7/kernel)]]

Caused by op 'block1_conv1_7/kernel/read', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mattia-lecci/venv_project/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mattia-lecci/venv_project/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mattia-lecci/venv_project/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/mattia-lecci/venv_project/lib/python3.5/s

In [24]:
X_train, Y_train, X_test, Y_test = data()
print('data loaded')

val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

Getting data


100%|██████████| 10/10 [00:00<00:00, 197.73it/s]


Splitting into training/validation
data loaded
3/3 [==============================] - 1s 187ms/step
Evalutation of best performing model:
Validation loss:  4.78766489029
Validation accuracy:  0.0


In [27]:
print(best_model.weights())

TypeError: 'list' object is not callable

In [ ]:
best_model.save(modelPath);
best_model.summary()

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)